In this model we will use two famous datasets for heartbeat classification:
* MIT-BIH Arrhythmia Dataset

The datasets contain signals that correspond to the shapes of heartbeats '\n'
obtained by an electrocardiogram (ECG). These signals are preprocessed \n
and segmented so that each segment corresponds to a beat.

**MIT-BIH Arrythmia Dataset**

    Number of Samples: 109446
    Number of Categories: 5
    Sampling Frequency: 125Hz
    Data Source: Physionet's MIT-BIH Arrhythmia Dataset
    Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]


### ECG-CLASSIFICATION OF SIGNALS

### Loading the Data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight,resample

In [3]:
train_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
tests_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv',header=None)

In [4]:
train_df

In [5]:
classes = train_df[187].value_counts()
print(classes)

In [6]:
plt.figure(figsize=(7,7))
explode = (0,0.1,0.1,0.1,0.1)
plt.pie(classes,labels=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'], autopct='%1.1f%%', explode=explode)
plt.show()

In [7]:
df_0 = train_df[train_df[187]==0.0].sample(n=16000, random_state=1)
df_1 = train_df[train_df[187]==1.0]
df_2 = train_df[train_df[187]==2.0]
df_3 = train_df[train_df[187]==3.0]
df_4 = train_df[train_df[187]==4.0]
# upsample
df_1_up=resample(df_1, replace=True, n_samples=16000, random_state=1)
df_2_up=resample(df_2, replace=True, n_samples=16000, random_state=1)
df_3_up=resample(df_3, replace=True, n_samples=16000, random_state=1)
df_4_up=resample(df_4, replace=True, n_samples=16000, random_state=1)

train_df = pd.concat([df_0, df_1_up, df_2_up, df_3_up, df_4_up])

In [8]:
train_df

In [9]:
classes = train_df[187].value_counts()
print(classes)

In [10]:
plt.figure(figsize=(7,7))
plt.pie(classes,labels=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'], autopct='%1.1f%%')
plt.show()

### We will analyze some samples of the classes

In [11]:
sample_n=df_0.sample(1)
sample_s=df_1.sample(1)
sample_v=df_2.sample(1)
sample_f=df_3.sample(1)
sample_q=df_4.sample(1)

fig,ax = plt.subplots(5,sharex=True,sharey=True)
fig.set_size_inches(9,9)

ax[0].plot(sample_n.iloc[0,:186], color='red')
ax[0].set_title("Normal beats")
ax[1].plot(sample_s.iloc[0,:186], color='red')
ax[1].set_title("Supraventricular ectopic beats")
ax[2].plot(sample_v.iloc[0,:186], color='red')
ax[2].set_title("Ventricular ectopic beats")
ax[3].plot(sample_f.iloc[0,:186], color='red')
ax[3].set_title("Fusion beats")
ax[4].plot(sample_q.iloc[0,:186], color='red')
ax[4].set_title("Unknown beats")
plt.xlabel('Time')
plt.ylabel('Magnitude')


### Data preprocessing

In [12]:
Y_train = to_categorical(train_df[187])
Y_tests = to_categorical(tests_df[187])

X_train = train_df.iloc[:,:186].values
X_train = X_train.reshape(len(X_train), X_train.shape[1],1)
X_tests = tests_df.iloc[:,:186].values
X_tests = X_tests.reshape(len(X_tests), X_tests.shape[1],1)

### Convolutional neural network

In [13]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import History 

history = History()
Model = Sequential()
Model.add(Conv1D(128,3,
                 input_shape=(X_train.shape[1],1),
                 activation='relu'))

Model.add(BatchNormalization())
Model.add(MaxPooling1D(pool_size=2))

Model.add(Conv1D(64,3, activation='relu'))
Model.add(BatchNormalization())
Model.add(MaxPooling1D(pool_size=2))

Model.add(Conv1D(64,2, activation='relu'))
Model.add(BatchNormalization())
Model.add(MaxPooling1D(pool_size=2))

Model.add(Conv1D(64,2, activation='relu'))

Model.add(Flatten())

Model.add(Dense(128, activation='relu'))

Model.add(Dropout(0.5))

Model.add(Dense(64,  activation='relu'))

Model.add(Dense(5,activation='softmax'))

### Training the Model

In [14]:
Model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=8), 
    ModelCheckpoint(filepath='./best_weights.h5', monitor='val_loss', save_best_only=True)
]

In [15]:
Model.summary()

In [16]:
eps = 12
bts = 32

history = Model.fit(X_train, Y_train, 
          epochs=eps, callbacks=callbacks,
          batch_size=bts, validation_data=(X_tests,Y_tests))

In [17]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Testing the Model

In [18]:
loss,accuracy = Model.evaluate(X_tests,Y_tests,verbose=1)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

y_predict = Model.predict(X_tests)

In [19]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [20]:
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(9, 9))
cf_matrix = confusion_matrix(Y_tests.argmax(axis=1),y_predict.argmax(axis=1))
plot_confusion_matrix(cf_matrix, 
                      classes=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'],
                      normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

### Save the model

In [21]:
Model.save('./ecg_model.h5')
Model.save_weights('./ecg_weights.h5')

### Performing prediction

In [22]:
def printPrediction(max_value):
    if(max_value==0):
        print("Non-Ectopic Beats")
    elif(max_value==1):
        print("Supraventricular Ectopic Beats")
    elif(max_value==2):
        print("Ventricular Ectopic Beats")
    elif(max_value==3):
        print("Fusion Beats")
    else:
        print("Unknown Beats")

In [23]:
my_list = []
prediction  = Model.predict(np.array([X_train[25000]])).tolist()

for it in prediction:
    for el in it:
        my_list.append(el)
        
print(my_list)        
max_value = my_list.index(max(my_list))

if(max_value==0):
    print("Normal Beats")
elif(max_value==1):
    print("Supraventricular Ectopic Beats")
elif(max_value==2):
    print("Ventricular Ectopic Beats")
elif(max_value==3):
    print("Fusion Beats")
else:
    print("Unknown Beats")

### ECG classification with noise

Train Neural Networks With Noise to Reduce Overfitting

In [24]:
gauss_std=0.5
gauss_k=0.1

# Gaussian noise in train_df
x_noise_train = train_df.iloc[:,:186].values.copy()
for i in range(len(x_noise_train)):
    x_noise_train[i,:186]=x_noise_train[i,:186]+gauss_k*np.random.normal(0,gauss_std,186)

x_noise_train_graph=x_noise_train
x_noise_train = x_noise_train.reshape(
    len(x_noise_train),
    x_noise_train.shape[1],1)

# Gaussian noise in tests_df
x_noise_tests = tests_df.iloc[:,:186].values.copy()
for i in range(len(x_noise_tests)):
    x_noise_tests[i,:186]=x_noise_tests[i,:186]+gauss_k*np.random.normal(0,gauss_std,186)

x_noise_tests_graph=x_noise_tests
x_noise_tests = x_noise_tests.reshape(
    len(x_noise_tests),
    x_noise_tests.shape[1],1)

In [25]:
loss,accuracy = Model.evaluate(x_noise_train,Y_train,verbose=1)

print('Noise Test loss:', loss)
print('Noise Test accuracy:', accuracy)

y_noise_predict = Model.predict(x_noise_train)

In [26]:
plt.figure(figsize=(9, 9))

cf_matrix_noise = confusion_matrix(Y_train.argmax(axis=1),y_noise_predict.argmax(axis=1))
plot_confusion_matrix(cf_matrix_noise,
                      classes=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'],
                      normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

In [27]:
plt.plot(train_df.iloc[0,:186], color='red')
plt.show()

In [28]:
plt.plot(x_noise_train_graph[0], color='red')
plt.show()

### Applying Transfer Learning

In [29]:
from keras.models import load_model

Model_transfer = load_model('./ecg_model.h5')
Model_transfer.load_weights('./ecg_weights.h5')

Model_transfer.summary()

In [30]:
Model_transfer.trainable = False
Model_transfer.pop()

In [32]:
Model2 = Sequential()
Model2.add(Model_transfer)
Model2.add(Dense(64, activation='relu'))
Model2.add(Dropout(0.25))
Model2.add(Dense(5, activation='softmax'))

Model2.summary()

In [33]:
Model2.compile(
    loss='categorical_crossentropy',
    optimizer='adam',metrics=['accuracy']
)

callbacks2 = [
    EarlyStopping(monitor='val_loss', patience=8), 
    ModelCheckpoint(filepath='./best_noise_weights.h5', monitor='val_loss', save_best_only=True)
]

In [34]:
Model2.fit(x_noise_train,Y_train,
           epochs=10, callbacks=callbacks2, 
           batch_size=32, validation_data=(x_noise_tests,Y_tests))

### Saving the Model2

In [35]:
Model2.save('./best_noise_model.h5')
Model2.load_weights('./best_noise_weights.h5')

### Performing Fine Turning

In [37]:
from tensorflow.keras.optimizers import Adam

callbacks3 = [
    EarlyStopping(monitor='val_loss', patience=8), 
    ModelCheckpoint(filepath='./best_noise_weights2.h5', monitor='val_loss', save_best_only=True)
]

Model_transfer.trainable = True

Model2.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(0.0001),
    metrics=['accuracy']
)

Model2.fit(x_noise_train,Y_train, 
           epochs=10,callbacks=callbacks3, 
           batch_size=32, validation_data=(x_noise_tests,Y_tests))

### Confusion matrix for noisy signals

In [38]:
y_noise_predict2 = Model2.predict(x_noise_tests)

plt.figure(figsize=(9,9))
cf_matrix_noise2 = confusion_matrix(Y_tests.argmax(axis=1),y_noise_predict2.argmax(axis=1))
plot_confusion_matrix(cf_matrix_noise2, 
                      classes=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'],
                      normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

### Confusion matrix for preprocessed signals

In [40]:
y_predict2 = Model2.predict(X_tests)

plt.figure(figsize=(9,9))
cf_matrix2 = confusion_matrix(Y_tests.argmax(axis=1),y_predict2.argmax(axis=1))
plot_confusion_matrix(cf_matrix2, 
                      classes=['Normal beats','Unknown beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion beats'],
                      normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()